# Trabalho 2 - MO430
## Patrick de Carvalho Tavares Rezende Ferreira - 175480

Abaixo fazemos a leitura dos arquivos de amostras fornecidos, a1 e b1.

In [0]:
# Fixando a semente para termos números aleatórios capazes de serem reproduzidos.
set.seed(1234)

a1=read.csv(file.path("/content/a1.csv"), header = FALSE)
a1=as.numeric(unlist(a1))

b1=read.csv(file.path("/content/b1.csv"), header = FALSE)
b1=as.numeric(unlist(b1))

### 1 - Intervalo de confiança

Abaixo utilizamos 3 testes para computar o intervalo de confiança da média do conjunto de valores a1 com 95% de confiança.

O primeiro teste utilizado é o "One Sample t-test", que usa um modelo paramétrico para estimar o intervalo de confiança (IC). O IC obtido é de [5.201134, 10.788916], com média de 7.995025. Dado o fato deste modelo fazer suposições acerca da origem dos dados (gaussiana), seu intervalo para o mesmo valor de confiaça é mais restrito que o teste de Wilcoxon, o que é desejável.

O segundo teste utilizado é o "Wilcoxon signed rank test", que usa um modelo não paramétrico para estimar o intervalo de confiança (IC). O IC obtido é de [4.887626, 11.054597], com pseudomediana de 7.98446. Dado o fato deste modelo não fazer suposições acerca da origem dos dados, seu intervalo para o mesmo valor de confiaça é mais amplo que o teste t, o que é não desejável mas ajuda em caso de não sabermos a natureza de origem dos dados.

O terceiro caso é a aplicação do bootstrap, uma técnica genérica baseada em amostragem (do conjunto de dados) para testes estatísticos e geração de intervalos de confiança. O IC obtido para 5000 repetições (bootstraps) em 95% de confiança BCa foi de [5.285, 10.334], sendo ainda mais restrito do que o próprio teste t.

Vale ressaltar que, nestas 3 aplicações de testes estatísticos, o que estamos calculando é a faixa de valores onde, com 95% de confiança, podemos encontrar a verdadeira média da fonte de dados que originou o dado conjunto de amostras. É por isso que não precisamos comparar mais de um conjunto de dados com outro, pois não estamos fazendo uma comparação entre eles.

In [18]:
set.seed(1234)
t.test(a1)

# Line separator
cat("\n\n------------------------------------------------------\n\n")

set.seed(1234)
wilcox.test(a1, conf.int = TRUE)

# Line separator
cat("\n\n------------------------------------------------------\n\n")


# Montando o calculo do bootstrap
library(boot)
data=read.csv(file.path("/content/a1.csv"), header = FALSE)
data = data$V1
foo <- function(data, indices){
  dt<-data[indices]
  c(
    mean(dt)
  )
}
set.seed(1234)
myBootstrap <- boot(data, foo, R=5000)
boot.ci(myBootstrap, index=1, type='bca')

# dim(iris)


	One Sample t-test

data:  a1
t = 6.1375, df = 14, p-value = 2.573e-05
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
  5.201134 10.788916
sample estimates:
mean of x 
 7.995025 




------------------------------------------------------




	Wilcoxon signed rank test

data:  a1
V = 120, p-value = 6.104e-05
alternative hypothesis: true location is not equal to 0
95 percent confidence interval:
  4.887626 11.054597
sample estimates:
(pseudo)median 
       7.98446 




------------------------------------------------------



BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 5000 bootstrap replicates

CALL : 
boot.ci(boot.out = myBootstrap, type = "bca", index = 1)

Intervals : 
Level       BCa          
95%   ( 5.285, 10.334 )  
Calculations and Intervals on Original Scale

### 2 - Intervalo de confiança para o tamanho de efeito

Abaixo computamos o tamanho de efeito Cohen D - pooled standard variation para os conjuntos a1 e b1. A estimativa média obtida para o tamanho de efeito é de 0.7400369, e o intervalo de confiança para esta é [0.0221951, 1.4578786].

In [19]:
# std_a1 = sd(a1)
# std_b1 = sd(b1)

# mean_a1 = mean(a1)
# mean_b1 = mean(b1)

# SDpooled = sqrt((std_a1^2 + std_b1^2)/2)

# cohen_D = (mean_a1 - mean_b1)/SDpooled

# print(cohen_D)

install.packages("devtools")  ## if not already installed
devtools::install_github("mtorchiano/effsize")

library(effsize)

# Line separator
cat("\n\n------------------------------------------------------\n\n")

set.seed(1234)
cohen.d(a1,b1, pooled = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Skipping install of 'effsize' from a github remote, the SHA1 (ae471a68) has not changed since last install.
  Use `force = TRUE` to force installation





------------------------------------------------------




Cohen's d

d estimate: 0.7400369 (medium)
95 percent confidence interval:
    lower     upper 
0.0221951 1.4578786 


### 3 - Intervalo de confiança para o tamanho de efeito

O intervalo de confiança computado para o Cohen D foi de [0.0221951, 1.4578786], com 95% de confiança (default do método por implementação). Este intervalo não contempla o valor zero, o que significa que, com 95% de confiança, podemos dizer que a diferença entre os conjuntos é significativa.

Entretanto, a comparação dos conjuntos a1 e b1 através do teste t resultou em um p-valor de 0.05724, acima de 0.05. Isto não significa que os conjuntos sejam de fontes iguais, apenas que o teste t não teve evidência o suficiente para garantir um p-valor menor que este, garde forma a garantir a evidência da diferença entre os conjuntos. 

Esta diferença entre as interpretações dos resultados destes dois métodos se deve, primeiramente, à diferença das estatístias utilizadas para o cálculo de cada um. Em segundo, o tamanho de efeito chegou bem perto de incluir o zero, o que indicaria a possibilidade dos conjuntos não serem estatísticamente diferentes, enquanto que o p-valor ficou também bem próximo de 0.05, o que indicaria uma provável diferença significativa entre os conjuntos. Ou seja, por uma margem numericamente pequena, os testes apontaram conclusões diferentes, o que indica que o Cohen D provavemente é um teste mais "forte" do que o teste t.

In [20]:
cat("\n\nIntervalo de confiança para o Cohen D: \n")
set.seed(1234)
cohen.d(a1,b1, pooled = TRUE)$conf.int

# Line separator
cat("\n\n------------------------------------------------------\n\n")

set.seed(1234)
t.test(a1, b1)



Intervalo de confiança para o Cohen D: 


lower     upper 
0.0221951 1.4578786



------------------------------------------------------




	Welch Two Sample t-test

data:  a1 and b1
t = 2.0123, df = 20.915, p-value = 0.05724
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.09871863  5.96056902
sample estimates:
mean of x mean of y 
 7.995025  5.064100 
